In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Import Libraries
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, Dataset

In [3]:
# Use GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
# df = pd.read_csv('dataset/training0_sort.csv')

In [5]:
# df.info()

In [6]:
# display datatypes
# df.dtypes

In [7]:
# df.dtypes.value_counts()

In [8]:
# run this
# df.fillna("0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5,#", inplace=True)

In [9]:
# df = pd.read_csv('dataset/training5_sort.csv')
# df.fillna("0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5,#", inplace=True)


In [10]:
# df

In [11]:

# df.iloc[1,1:].values
# type(df.iloc[1,:].values)

In [4]:
# Our custom Dataset object
class RubiksDataset(Dataset):
    def __init__(self, df):
        self.df = df
        move_dict = {"#":0, "U":1, "U'":2, "U2":3, "D":4, "D'":5, "D2":6, "L":7, "L'":8, "L2":9, "R":10, 
                     "R'":11, "R2":12, "F":13, "F'":14, "F2":15, "B":16, "B'":17, "B2":18}
        # list of list, where inner list is an entire sequence.
        self.states = [] #is now a list of list of list :D
        self.moves = [] #list of list

        for i in range(len(df)):
            statelist = []
            movelist = []
            statemoves = df.iloc[i,1:]
            for line in statemoves:
                split = line.split(",")
                # state_as_int = int(split[0].replace(" ",""))/(10**54)
                state_as_str_list = split[0].split(" ")
                state_as_int_list = [int(i) for i in state_as_str_list]
                # above sets the input states to int values instead of string
                # below sets the string move into an int value,
                move_as_int = move_dict.get(split[1])

                statelist.append(state_as_int_list)
                movelist.append(move_as_int)
            self.states.append(statelist)
            self.moves.append(movelist)


        
    def __len__(self):
        return len(self.moves)
        
    def __getitem__(self, index):
        # inputs = torch.tensor(self.inputs[index]).float()
        # outputs = torch.tensor(self.outputs[index]).float()
        # print(self.states[index])
        inputs = torch.tensor(self.states[index]) # list of list, each index is a state vector
        outputs = torch.tensor(self.moves[index]) # list, each index is the move set of that sequence(index)
        return inputs, outputs

In [13]:
# dataset = RubiksDataset(df)
# # dataloader = DataLoader(dataset, batch_size = 1, shuffle = False)
# dataloader = DataLoader(dataset, batch_size = 128, shuffle = True)

In [14]:
# df = pd.read_csv('dataset/training5_sort.csv')
# df.fillna("0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5,#", inplace=True)
# dataset = RubiksDataset(df)
# # dataloader = DataLoader(dataset, batch_size = 1, shuffle = False)
# dataloader = DataLoader(dataset, batch_size = 128, shuffle = True)

In [15]:
# seqcount = 0
# for inputs, outputs in dataloader:
#     seqcount +=1
#     print(outputs)
#     # for i in outputs:
#     #     print(i)
#     print(seqcount)

In [5]:
class RNN(torch.nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.layers = torch.nn.Sequential(torch.nn.Linear(108, 256),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(256, 256),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(256, 128),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(128, 73))
        
    def forward(self, inputs, hidden):
        # combined = torch.tensor([inputs, hidden]).to(inputs.device)
        # print(inputs.shape)
        # print(hidden.shape)
        combined = torch.cat([inputs, hidden]).to(inputs.device)
        out = self.layers(combined)
        return out

In [9]:
# model = RNN().to(device)
# print(model.layers)

Sequential(
  (0): Linear(in_features=108, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=73, bias=True)
)


In [6]:
def train(model, dataloader, num_epochs, learning_rate, device):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    # hidden = torch.tensor([0]).to(device)
    # hidden= torch.zeros(1, 81, 54).to(device)
    for epoch in range(num_epochs):
        count = 0
        loss = 0

        
        # optimizer.zero_grad()
        for inputs, targets in dataloader:
            # inputs = inputs.permute(1,0,2)
            # need another loop inside.
            correct = 0
            pred = 0
            
            # reset hidden here
            loss = 0
            optimizer.zero_grad()
            # 
            for i in range(len(inputs)):
                hidden= torch.zeros(54).to(device)
                for j in range(len(inputs[i])):
                    outputs = model(inputs[i][j].to(device), hidden)
                    out, hidden = outputs[0:19], outputs[19::]
                    # print(out.shape)
                    # print(hidden.shape)
                    # print(targets[i][j].shape)

                    # print((torch.argmax(out) == targets[i][j]).item())
                    correct += (torch.argmax(out) == targets[i][j]).item()
                    pred += 1
                    loss += criterion(out.to(device), targets[i][j].to(device)) / len(inputs[i]) / len(inputs)
                # print(loss)
            batch_accuracy = correct/pred
            loss.backward()
            optimizer.step()
            count +=1
            print(f"Batch number: {count}")
            print(f"Batch {count} accuracy: {batch_accuracy}")
        
        # loss /= len(inputs)
        # loss.backward()
        # optimizer.step()

        #     outputs = model(inputs.to(device), hidden)
        #     out, hidden = outputs[0], outputs[1]
        #     loss += criterion(out.to(device), targets.to(device))
        # loss /= len(dataloader)
        # loss.backward()
        # optimizer.step()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Last Batch Accuracy: {batch_accuracy*100:.2f}%")

In [7]:
model = RNN().to(device)
model.load_state_dict(torch.load('weights/rnn_4layer_256_37.pth'))

<All keys matched successfully>

In [19]:
df = pd.read_csv('dataset/training0_sort.csv')
df.fillna("0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5,#", inplace=True)
dataset = RubiksDataset(df)
# dataloader = DataLoader(dataset, batch_size = 1, shuffle = False)
dataloader = DataLoader(dataset, batch_size = 128, shuffle = True)

C:\Users\samje\AppData\Local\Temp\ipykernel_19860\299646355.py:1: DtypeWarning: Columns (68,69,70,71,72,73,74,75,76,77,78,79,80,81) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dataset/training0_sort.csv')


In [20]:
# Train the model
# model = RNN().to(device)
train(model, dataloader, num_epochs = 1, learning_rate = 0.0001, device = device)

Batch number: 1
Batch 1 accuracy: 0.7511574074074074
Batch number: 2
Batch 2 accuracy: 0.7559799382716049
Batch number: 3
Batch 3 accuracy: 0.7527970679012346
Batch number: 4
Batch 4 accuracy: 0.7520254629629629
Batch number: 5
Batch 5 accuracy: 0.7524112654320988
Batch number: 6
Batch 6 accuracy: 0.7592592592592593
Batch number: 7
Batch 7 accuracy: 0.7594521604938271
Batch number: 8
Batch 8 accuracy: 0.7555941358024691
Batch number: 9
Batch 9 accuracy: 0.7546296296296297
Batch number: 10
Batch 10 accuracy: 0.7541473765432098
Batch number: 11
Batch 11 accuracy: 0.7568479938271605
Batch number: 12
Batch 12 accuracy: 0.7546296296296297
Batch number: 13
Batch 13 accuracy: 0.7543402777777778
Batch number: 14
Batch 14 accuracy: 0.7554012345679012
Batch number: 15
Batch 15 accuracy: 0.7535686728395061
Batch number: 16
Batch 16 accuracy: 0.7510609567901234
Batch number: 17
Batch 17 accuracy: 0.7588734567901234
Batch number: 18
Batch 18 accuracy: 0.7546296296296297
Batch number: 19
Batch 19 ac

In [21]:
# Save the model use new names each time tyvm
torch.save(model.state_dict(), 'weights/rnn_4layer_256_x.pth')

In [24]:
df = pd.read_csv('dataset/training1_sort.csv')
df.fillna("0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5,#", inplace=True)
dataset = RubiksDataset(df)
# dataloader = DataLoader(dataset, batch_size = 1, shuffle = False)
dataloader = DataLoader(dataset, batch_size = 128, shuffle = True)

C:\Users\samje\AppData\Local\Temp\ipykernel_3552\1325915394.py:1: DtypeWarning: Columns (69,70,71,72,73,74,75,76,77,78,79,80,81,82) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dataset/training7_sort.csv')


In [ ]:
train(model, dataloader, num_epochs = 2, learning_rate = 0.0005, device = device)

In [26]:
# Save the model use new names each time tyvm
torch.save(model.state_dict(), 'weights/rnn_4layer_256_x.pth')

In [27]:
df = pd.read_csv('dataset/training8_sort.csv')
df.fillna("0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5,#", inplace=True)
dataset = RubiksDataset(df)
# dataloader = DataLoader(dataset, batch_size = 1, shuffle = False)
dataloader = DataLoader(dataset, batch_size = 128, shuffle = True)

C:\Users\samje\AppData\Local\Temp\ipykernel_3552\1852383510.py:1: DtypeWarning: Columns (69,70,71,72,73,74,75,76,77,78,79,80,81) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dataset/training8_sort.csv')


In [ ]:
train(model, dataloader, num_epochs = 2, learning_rate = 0.0005, device = device)

In [29]:
# Save the model use new names each time tyvm
torch.save(model.state_dict(), 'weights/rnn_4layer_256_24.pth')

In [ ]:
# trainhere

In [61]:
df = pd.read_csv('dataset/training9_sort.csv')
df.fillna("0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5,#", inplace=True)
dataset = RubiksDataset(df)
# dataloader = DataLoader(dataset, batch_size = 1, shuffle = False)
dataloader = DataLoader(dataset, batch_size = 128, shuffle = True)

C:\Users\samje\AppData\Local\Temp\ipykernel_19860\52702347.py:1: DtypeWarning: Columns (69,70,71,72,73,74,75,76,77,78,79,80,81) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dataset/training9_sort.csv')


In [62]:
train(model, dataloader, num_epochs = 1, learning_rate = 0.0001, device = device)

Batch number: 1
Batch 1 accuracy: 0.7663966049382716
Batch number: 2
Batch 2 accuracy: 0.7642746913580247
Batch number: 3
Batch 3 accuracy: 0.7643711419753086
Batch number: 4
Batch 4 accuracy: 0.7601273148148148
Batch number: 5
Batch 5 accuracy: 0.7565586419753086
Batch number: 6
Batch 6 accuracy: 0.761766975308642
Batch number: 7
Batch 7 accuracy: 0.7588734567901234
Batch number: 8
Batch 8 accuracy: 0.7650462962962963
Batch number: 9
Batch 9 accuracy: 0.761766975308642
Batch number: 10
Batch 10 accuracy: 0.7645640432098766
Batch number: 11
Batch 11 accuracy: 0.7602237654320988
Batch number: 12
Batch 12 accuracy: 0.7684220679012346
Batch number: 13
Batch 13 accuracy: 0.7666859567901234
Batch number: 14
Batch 14 accuracy: 0.7611882716049383
Batch number: 15
Batch 15 accuracy: 0.759741512345679
Batch number: 16
Batch 16 accuracy: 0.7641782407407407
Batch number: 17
Batch 17 accuracy: 0.7660108024691358
Batch number: 18
Batch 18 accuracy: 0.7643711419753086
Batch number: 19
Batch 19 accur

In [63]:
# Save the model use new names each time tyvm
torch.save(model.state_dict(), 'weights/rnn_4layer_256_37.pth')

In [33]:
# df = pd.read_csv('dataset/training6_sort.csv')
# df.fillna("0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5,#", inplace=True)
# dataset = RubiksDataset(df)
# # dataloader = DataLoader(dataset, batch_size = 1, shuffle = False)
# dataloader = DataLoader(dataset, batch_size = 128, shuffle = True)

In [34]:
# # Save the model use new names each time tyvm
# torch.save(model.state_dict(), 'weights/rnn_4layer_256_18.pth')

In [35]:
# # Load the model
# model = RNN().to(device)
# model.load_state_dict(torch.load('weights/rnn_4layer_256_14.pth'))
# # model.load_state_dict(torch.load('weights/rnn_4layer_256_6.pth', map_location=torch.device('cpu')))


In [8]:
test_dataset = pd.read_csv("dataset/training99_sort.csv")
test_dataset.fillna("0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5,#", inplace=True)
test_rubiksdataset = RubiksDataset(test_dataset)
test_dataloader = DataLoader(test_rubiksdataset, batch_size = 128)

C:\Users\samje\AppData\Local\Temp\ipykernel_1812\414264106.py:1: DtypeWarning: Columns (69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  test_dataset = pd.read_csv("dataset/training99_sort.csv")


In [23]:
# def test(model, test_loader, device):
#     model.eval()
#     # test_loss = 0
#     correct = 0
#     pred = 0
#     count = 0
#     sumofaccuracy = 0
#     # criterion = torch.nn.CrossEntropyLoss()
#     with torch.no_grad():
#         for data, target in test_loader:
#             for i in range(len(data)):
#                 hidden= torch.zeros(54).to(device)
#                 for j in range(len(data[i])):
#                     output = model(data[i][j].to(device), hidden)
#                     out, hidden = output[0:19], output[19::]
#                     # test_loss += criterion(out.to(device), target[i][j].to(device)) / len(data[i]) / len(data)
#                     # pred = out.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
#                     # correct += pred.eq(target.view_as(pred)).sum().item()
#                     # if args.dry_run:
#                     #     break
#                     correct += (torch.argmax(out) == target[i][j]).item()
#                     pred += 1
#             batch_accuracy = correct/pred

#     # test_loss /= len(test_loader.dataset)

#     # print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
#     #     test_loss, correct, len(test_loader.dataset),
#     #     100. * correct / len(test_loader.dataset)))
#             count +=1
#             print(f"Batch {count} accuracy: {batch_accuracy}")
#             sumofaccuracy += batch_accuracy
#     print(f"Total Test accuracy: {sumofaccuracy/count} , for {count} batch in total.")

In [13]:
def test(model, test_loader, device):
    model.eval()
    # test_loss = 0
    correct = 0
    pred = 0
    count = 0
    sumofaccuracy = 0

    # Number of times a move was predicted as the following and was correct
    correct_pred_move_count = {"#": 0, "U": 0, "U'": 0, "U2": 0, "D": 0, "D'": 0, "D2": 0, "L": 0, "L'": 0, "L2": 0, "R": 0, 
                     "R'": 0, "R2": 0, "F": 0, "F'": 0, "F2": 0, "B": 0, "B'": 0, "B2": 0}
    
    # Number of times this move was the correct move
    true_move_count = {"#": 0, "U": 0, "U'": 0, "U2": 0, "D": 0, "D'": 0, "D2": 0, "L": 0, "L'": 0, "L2": 0, "R": 0, 
                     "R'": 0, "R2": 0, "F": 0, "F'": 0, "F2": 0, "B": 0, "B'": 0, "B2": 0}

    move_mapping = ["#", "U", "U'", "U2", "D", "D'", "D2", "L", "L'", "L2", "R", 
                     "R'", "R2", "F", "F'", "F2", "B", "B'", "B2"]

    # criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        for data, target in test_loader:
            for i in range(len(data)):
                hidden= torch.zeros(54).to(device)
                for j in range(len(data[i])):
                    output = model(data[i][j].to(device), hidden)
                    out, hidden = output[0:19], output[19::]
                    # test_loss += criterion(out.to(device), target[i][j].to(device)) / len(data[i]) / len(data)
                    # pred = out.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                    # correct += pred.eq(target.view_as(pred)).sum().item()
                    # if args.dry_run:
                    #     break
                    pred_move = torch.argmax(out)
                    correct += (pred_move == target[i][j]).item()
                    pred += 1

                    if (pred_move == target[i][j]).item():
                        correct_pred_move_count[move_mapping[torch.argmax(out).item()]] += 1
                    true_move_count[move_mapping[target[i][j].item()]] += 1
            # print(correct_pred_move_count)
            # print(true_move_count)
            batch_accuracy = correct/pred

    # test_loss /= len(test_loader.dataset)

    # print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    #     test_loss, correct, len(test_loader.dataset),
    #     100. * correct / len(test_loader.dataset)))
            count +=1
            print(f"Batch {count} accuracy: {batch_accuracy}")
            sumofaccuracy += batch_accuracy
            
    print(f"Total Test accuracy: {sumofaccuracy/count} , for {count} batch in total.")
    true_prediction_accuracy_dict = {move: f'{correct_pred_move_count[move]/true_move_count[move]*100:.2f}%' for move in move_mapping}
    print(f"True prediction accuracies: {true_prediction_accuracy_dict}")
    print(correct_pred_move_count)
    print(true_move_count)

In [14]:
test(model, test_dataloader, device=device)


Batch 1 accuracy: 0.8017113095238095
Batch 2 accuracy: 0.7994791666666666
Batch 3 accuracy: 0.7979910714285714
Batch 4 accuracy: 0.7975260416666666
Batch 5 accuracy: 0.7967447916666667
Batch 6 accuracy: 0.7960844494047619
Batch 7 accuracy: 0.7960512329931972
Batch 8 accuracy: 0.7955845424107143
Batch 9 accuracy: 0.7946118551587301
Batch 10 accuracy: 0.7941313244047619
Batch 11 accuracy: 0.793171672077922
Batch 12 accuracy: 0.7927207341269841
Batch 13 accuracy: 0.792167467948718
Batch 14 accuracy: 0.7915869472789115
Batch 15 accuracy: 0.7910280257936508
Batch 16 accuracy: 0.7906552269345238
Batch 17 accuracy: 0.790063681722689
Batch 18 accuracy: 0.7897393766534392
Batch 19 accuracy: 0.7893317277568922
Batch 20 accuracy: 0.7888997395833334
Batch 21 accuracy: 0.7886506164965986
Batch 22 accuracy: 0.7882846320346321
Batch 23 accuracy: 0.7880556094720497
Batch 24 accuracy: 0.7875279017857143
Batch 25 accuracy: 0.7870833333333334
Batch 26 accuracy: 0.7868267799908425
Batch 27 accuracy: 0.786